In [7]:
from retrieve import retrieve
vector_store_path = 'vector_store'
EB_PATH = r'D:\Huan\Project\KHDL\model\ebmodel\bge-m3-ft-triplet'
re = retrieve(vector_store_path = vector_store_path, EMBEDDING_MODEL_PATH = EB_PATH)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\importlib\__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


Load embedding model...  Done
Load vector store...  Done


In [19]:
query = 'Triệu chứng của bệnh loạn dưỡng cơ Duchenne?'
docs = re.similarity_search(query=query,top_k=10)


In [20]:
docs = [i.page_content for i in docs]

In [21]:
print(docs)

['Triệu chứng', 'Loạn dưỡng cơ (tiếng Anh: Muscular dystrophy) đề cập đến một nhóm bệnh cơ liên quan đến gen, các bệnh này làm yếu các cơ trong thể người. Loạn dưỡng cơ đặc trưng bởi một quá trình làm yếu các cơ xương do thiếu hụt các protein cơ, và sự chết đi của các tế bào và mô. Có 9 bệnh được xếp vào nhóm  này gồm Duchenne, Becker, thắt lưng-vai,\nbẩm sinh, facioscapulohumeral, tăng trương lực cơ, oculopharyngeal, ngoại biên, và Emery-Dreifuss mặc dù, tổng số có hơn 100 bệnh tương tự như bệnh này. Hầu hết các bệnh MD liên quan đến rối loạn đa hệ thống trong cơ thể như hệ tim mạch, hệ tiêu hóa, và hệ thần kinh, các tuyến nội tiết, da, mắt và thậm chí ở não. Tình trạng này cũng có thể dẫn đến các thay đổi về tâm trạng và khó khăn trong việc học.\n Chẩn đoán \nViệc chẩn đoán bệnh dựa trên các kết quả sinh thiết cơ và sự gia tăng creatine phosphokinase (CpK3). Trong một số trường hợp, thí nghiêm DNA tế bào máu có thể đủ yêu cầu.', 'Triệu chứng \nDMD là một kiểu bệnh loạn dưỡng cơ nghiê

In [22]:
# Đường dẫn mô hình đã lưu
load_path = r"D:\Huan\Project\KHDL\reranker"

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(load_path)
model = AutoModelForSequenceClassification.from_pretrained(load_path)
model.eval()



Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at D:\Huan\Project\KHDL\reranker and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_f

In [23]:
pairs = [[query, doc] for doc in docs]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=1024)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([-0.2844, -0.1790, -0.1131, -0.1713, -0.2281, -0.1313, -0.2224, -0.1789,
        -0.2329, -0.2480])


In [24]:
scores.argmax()

tensor(2)

In [25]:
print(docs[2])

Triệu chứng 
DMD là một kiểu bệnh loạn dưỡng cơ nghiêm trọng nhất. Triệu chứng  thường bắt đầu ở bé trai khoảng 3 - 4 tuổi biểu hiện ở yếu cơ và teo cơ, ban đầu thường ở các cơ đùi, cơ bắp chân, sau đó là ở cơ cánh tay. Vì thế người bệnh đi lại và đứng lên rất khó khăn. Đến tuổi 12 thường không thể tự đi bộ. Ngoài ra nhiều người bệnh còn bị kèm chứng vẹo cột sống, một số còn bị thiểu năng trí tuệ. Nữ giới cũng có thể biểu hiện một số triệu chứng này, nhưng tần số bị bệnh rất thấp và mức độ biểu hiện bệnh thường nhẹ hơn. Nói chung, tuổi thọ trung bình của người bệnh là 26, nếu chăm sóc thật đầy đủ thì một số có thể sống tới độ tuổi 30 hoặc 40.
Bệnh thường gặp ở nam, xảy ra với tần số khoảng 1/3600 trẻ sơ sinh nam, có tiền sử gia đình mắc bệnh này.
Ngoài các triệu chứng dễ nhận nói trên, người bệnh còn có thể có các biểu hiện sau:
-     Nếu đi được thì thường có xu hướng đi bằng bàn chân trước, hoặc thậm chí đi trên các ngón chân;
-     Hay bị ngã, thường mỏi cơ;
